In [1]:
import numpy                 as np
import pandas                as pd
import glob                  as g
from   datetime              import date

from pyspark.sql             import SparkSession
from pyspark.sql.types       import StringType
from pyspark.sql.types       import TimestampType
from pyspark.sql             import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions   import *
from pyspark.sql             import Window
from pyspark.sql.types       import *
#from  pyspark.sql.functions import input_file_name

spark = (
    SparkSession
        .builder.master('local[*]')
        .appName('Churn')
        .config('spark.driver.memory','2g') 
        .config('spark.executor.memory','2g')
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .config('spark.ui.showConsoleProgress', True)
        .config("viewsEnabled", "true")
        .config("spark.sql.debug.maxToStringFields", 1000)
        .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/20 08:07:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
customSchema = StructType([
                           StructField("Index", StringType(), True),
                           StructField("Date", DateType(), True), 
                           StructField("IBGE_ID", StringType(), True),
                           StructField("dewpoint_temperature", DoubleType(), True), 
                           StructField("humidity", DoubleType(), True),
                           StructField("max_temperature", DoubleType(), True), 
                           StructField("min_temperature", DoubleType(), True),
                           StructField("surface_pressure", DoubleType(), True),
                           StructField("temperature", DoubleType(), True),
                           StructField("total_precipitation", DoubleType(), True), 
                           StructField("u_component_of_wind_10m", DoubleType(), True),
                           StructField("v_component_of_wind_10m", DoubleType(), True),
                           StructField("geo", DoubleType(), True),
                          ])

In [3]:
df = (spark.read.format("csv") 
      .option("header", "True") 
      #.schema(customSchema) 
      .load('/Users/aurelianosancho/Documents/Agro/raw_data/*.csv') 
      .withColumn("filename", input_file_name())
    )

df = df.select(
                'Date', 'RegionID', 'dewpoint_temperature_2m',
                'max_temperature_2m', 'min_temperature_2m','temperature_2m',
                'humidity',  'surface_pressure', 'total_precipitation',
                'u_component_of_wind_10m', 'v_component_of_wind_10m', 
                )

# df.drop_duplicates(subset=['Date']) # Não funciona

In [4]:
(
    df.write.format("parquet")
    .mode("overwrite")
    .save("clima_Agro.parquet")
)

In [ ]:
fim